In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-02-16 07:27:52--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2020-02-16 07:27:52 (8.10 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [0]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [0]:
import pandas as pd
from IPython.display import display

download_dir = './ml-latest-small'
links = pd.read_csv(download_dir + '/links.csv')
display(links.head())
tag = pd.read_csv(download_dir + '/tags.csv')
display(tag.head())
ratings = pd.read_csv(download_dir + '/ratings.csv')
display(ratings.head())
movies = pd.read_csv(download_dir + '/movies.csv')
movies.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


For this dataset, the movies are already clustered into genres, and the time of ratings are given. Both are useful information, the former can give a direction to building the movie embeddings, the latter can tell us how the user profile has changed over years. Therefore, to take into account of these factors, we assume rating is a function of user embedding (numeric representation of user profile), time, movie embedding and genres. To build a baseline model, we try to incorporate these ideas into the architecture.

In [0]:
# check if any user didnt give a rating
ratings.rating.isnull().value_counts()

False    100836
Name: rating, dtype: int64

In [0]:
# left join genres onto ratings table
df = ratings.join(movies.set_index('movieId'), on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [0]:
# turn timestamp from seconds to year
change_to_year = lambda x: pd.Timestamp(x, unit='s').year
df.timestamp = df.timestamp.apply(change_to_year)
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [30]:
# turn genres into vector representation
genres_list = df.genres.tolist()
genres = []

for genre in genres_list:
  genres.extend(genre.split('|'))

genres = set(genres)
print('unique genres: {}'.format(len(genres)))

unique genres: 20


In [0]:
# form a dictionary from set and encode genres into a 